In [14]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...


In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


In [10]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [15]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [16]:
pickle.dump(words,open('words1.pkl','wb'))
pickle.dump(classes,open('classes1.pkl','wb'))

In [17]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


<ipython-input-17-14f4db91aa1f>:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [18]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model1.h5', hist)

print("model created")


C:\Users\HP\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
10/10 [==============================] - 1s 8ms/step - loss: 2.1830 - accuracy: 0.1702
Epoch 2/200
10/10 [==============================] - 0s 5ms/step - loss: 2.0600 - accuracy: 0.2553
Epoch 3/200
10/10 [==============================] - 0s 4ms/step - loss: 2.0924 - accuracy: 0.1915
Epoch 4/200
10/10 [==============================] - 0s 5ms/step - loss: 1.9744 - accuracy: 0.3404
Epoch 5/200
10/10 [==============================] - 0s 4ms/step - loss: 1.7803 - accuracy: 0.4043
Epoch 6/200
10/10 [==============================] - 0s 4ms/step - loss: 1.7186 - accuracy: 0.4681
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 1.5417 - accuracy: 0.6170
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.4861 - accuracy: 0.5532
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.4149 - accuracy: 0.5745
Epoch 10/200
10/10 [==============================] - 0s 4ms/step - loss: 1.2365 - accuracy: 0.5957
Epoch 11/

10/10 [==============================] - 0s 3ms/step - loss: 0.0267 - accuracy: 1.0000
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0319 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0776 - accuracy: 0.9574
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0648 - accuracy: 0.9787
Epoch 87/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0323 - accuracy: 1.0000
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.1693 - accuracy: 0.9362
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0649 - accuracy: 0.9574
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0385 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0129 - accuracy: 1.0000
Epoch 92/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0597 - accuracy: 1.0000
Epoch 93/200


10/10 [==============================] - 0s 4ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0297 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0456 - accuracy: 0.9787
Epoch 167/200
10/10 [==============================] - 0s 5ms/step - loss: 0.0178 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0232 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0374 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0110 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0262 - accuracy: 1.0000
Epoch 173/200
10/10 [==============================] - 0s 4ms/step - loss: 0.0425 - accuracy: 0.9787
Epoc